In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base
from utils import read_api_credentials, get_sqlite_db_connection, close_sqlite_db_connection





# The API endpoint for retrieving accounts
accounts_url = "https://api.tempo.io/4/"

In [2]:
# AUTHENTICATION
api_credentials = read_api_credentials("config.ini", "api_tempo")
api_token = api_credentials['token']
endpoint1 = 'accounts'

headers = {
    'Authorization': f'Bearer {api_token}'
}

accounts_url = f"https://api.tempo.io/4/{endpoint1}"

Sections available: ['api_tempo', 'api_jira']


In [52]:
def df_accounts(endpoint):
    api_url = f'https://api.tempo.io/core/3/{endpoint}'
    response = requests.get(api_url, headers=headers)
    print(f'Status code: {response.status_code}')
    
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['results'])
        print("Columns After Normalization:", list(df.columns))
        
        # Define columns of interest, including the original 'links.self'
        selected_columns = ['id', 'key', 'name', 'status', 'lead.accountId', 'category.id', 'customer.name', 'links.self']
        df = df[selected_columns]
        
        # Rename columns for clarity
        rename_columns = {
            'lead.accountId': 'leadId',
            'category.id': 'categoryId',
            'customer.name': 'customerId',
            'links.self': 'accountLink'  # Use 'accountLink' for clarity
        }

        df = df.rename(columns=rename_columns)
        print("Columns after Filter and Renaming: ", list(df.columns))
        # Apply filtering for "OPEN" accounts and limit to the first 5 entries
        df_open = df[df['status'] == "OPEN"].head(5)

        print("Filtered OPEN accounts, limited to first 5 entries:")
        print(df_open)  # This will print the filtered and limited DataFrame
                
        # Filter for OPEN accounts and limit to the first 5 entries
        return df

    else:
        print(f"Failed to fetch accounts: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

# Call df_accounts function with the parameter endpoint1 to get accounts data from JIRA API.
accounts_df = df_accounts(endpoint1)
print(accounts_df.head())

Status code: 200
Columns After Normalization: ['self', 'key', 'id', 'name', 'status', 'global', 'monthlyBudget', 'lead.self', 'lead.accountId', 'lead.displayName', 'category.self', 'category.key', 'category.id', 'category.name', 'category.type.name', 'customer.self', 'customer.key', 'customer.id', 'customer.name', 'links.self', 'contact.self', 'contact.accountId', 'contact.displayName', 'contact.type']
Columns after Filter and Renaming:  ['id', 'key', 'name', 'status', 'leadId', 'categoryId', 'customerId', 'accountLink']
Filtered OPEN accounts, limited to first 5 entries:
    id                     key  \
1  474        LASMARIAS-SEC-03   
2  196  STARTIT-AND-TYCMANAGER   
3  183     AYSA-AND-WORDPRESS1   
4  212          EDENRED-ABS-01   
5  165           BCOCOM-INO-05   

                                                name status  \
1  LASMARIAS | SEC | Analisis de Vulnerabilidad 2021   OPEN   
2                               STARTIT | TyCManager   OPEN   
3                          

In [54]:
open_accounts_df = accounts_df[accounts_df['status'] == "OPEN"].head(5)

# Initialize an empty list for project IDs
project_ids = []

# Loop through each open account's link to fetch the project ID
for account_link in open_accounts_df['accountLink']:
    link_response = requests.get(account_link, headers=headers)
    link_data = link_response.json()
    
    # Check if 'results' is not empty and contains 'scope' with type "PROJECT"
    if link_data['results'] and link_data['results'][0]['scope']['type'] == "PROJECT":
        project_id = link_data['results'][0]['scope']['id']
        project_ids.append(project_id)
    else:
        project_ids.append(None)  # Append None if no project ID is found or if the structure is unexpected

# Assuming open_accounts_df is a copy or you're okay modifying it directly
open_accounts_df['projectId'] = project_ids
# Optionally, drop the 'accountLink' column if it's no longer needed
open_accounts_df.drop(columns=['accountLink'], inplace=True)

print(open_accounts_df)

    id                     key  \
1  474        LASMARIAS-SEC-03   
2  196  STARTIT-AND-TYCMANAGER   
3  183     AYSA-AND-WORDPRESS1   
4  212          EDENRED-ABS-01   
5  165           BCOCOM-INO-05   

                                                name status  \
1  LASMARIAS | SEC | Analisis de Vulnerabilidad 2021   OPEN   
2                               STARTIT | TyCManager   OPEN   
3                             AySA | AnD | WordPress   OPEN   
4                        EDENRED | ABS | Soporte CRM   OPEN   
5                               BCOCOM | InO | Abono   OPEN   

                     leadId  categoryId                          customerId  \
1  5ae1b3757dd1e556227bb11f        13.0                          Las Marias   
2  5a5f6fb94ec8fa28e08c63ed        15.0                            Start IT   
3  5a5f6fb94ec8fa28e08c63ed        11.0  Agua y Saneamientos Argentinos S.A   
4  5b198d4827de866101aacfd6        12.0                                 NaN   
5  5a5f42adbe08e62844